## Projeto: Sistema de Recomendação

## Análise Musical - Respondendo perguntas:

1. Quais são os top 10 artistas?
2. Qual o gênero musical desses top 10 artistas?
3. Quais são os top gêneros?
4. Quais são os artistas que estão nesses top gêneros?

In [0]:
# consultando dados disponíveis no diretório
display(dbutils.fs.ls('dbfs:/FileStore/tables/dados_spotify_tratados/'))

path,name,size,modificationTime
dbfs:/FileStore/tables/dados_spotify_tratados/data.parquet/,data.parquet/,0,0
dbfs:/FileStore/tables/dados_spotify_tratados/data_artist.parquet/,data_artist.parquet/,0,0
dbfs:/FileStore/tables/dados_spotify_tratados/data_genres.parquet/,data_genres.parquet/,0,0
dbfs:/FileStore/tables/dados_spotify_tratados/data_w_genres.parquet/,data_w_genres.parquet/,0,0
dbfs:/FileStore/tables/dados_spotify_tratados/data_year.parquet/,data_year.parquet/,0,0


In [0]:
# importação da biblioteca Pandas do Spark
import pyspark.pandas as ps

In [0]:
# realizando a leitura dos arquivos .parquet
df_genres = ps.read_parquet('dbfs:/FileStore/tables/dados_spotify_tratados/data_genres.parquet/')
df_artists = ps.read_parquet('dbfs:/FileStore/tables/dados_spotify_tratados/data_artist.parquet/')
df_w_genres = ps.read_parquet('dbfs:/FileStore/tables/dados_spotify_tratados/data_w_genres.parquet/')

In [0]:
# realizando leitura inicial dos 3 arquivos
df_genres.head(2)

,mode,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key
0,1,21st century classical,0.979333,0.162883,1.602977e+05,0.071317,0.606834,0.3616,-31.514333,0.040567,75.336500,0.103783,27.833333,6
1,1,432hz,0.494780,0.299333,1.048887e+06,0.450678,0.477762,0.1310,-16.854000,0.076817,120.285667,0.221750,52.500000,5


In [0]:
df_artists.head(2)

,mode,count,acousticness,artists,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key
0,1,9,0.590111,Cats 1981 Original London Cast,0.467222,250318.555556,0.394003,0.011400,0.290833,-14.448,0.210389,117.518111,0.389500,38.333333,5
1,1,26,0.862538,Cats 1983 Broadway Cast,0.441731,287280.000000,0.406808,0.081158,0.315215,-10.690,0.176212,103.044154,0.268865,30.576923,5


In [0]:
df_w_genres.head(2)

,genres,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count
0,show tunes,Cats 1981 Original London Cast,0.5901111111111111,0.4672222222222222,250318.5555555556,0.3940033333333333,0.011399851111111107,0.2908333333333333,-14.448,0.210389,117.518111,0.389500,38.333333,5.0,1.0,9.0
1,,Cats 1983 Broadway Cast,0.8625384615384617,0.4417307692307693,287280.0,0.4068076923076923,0.08115826423076923,0.3152153846153846,-10.690,0.176212,103.044154,0.268865,30.576923,5.0,1.0,26.0


**1. Quais são os top 10 artistas?**

In [0]:
# analisando dataframe com as variáveis de interesse para responder pergunta
df_artists.head()

,mode,count,acousticness,artists,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key
0,1,9,0.590111,Cats 1981 Original London Cast,0.467222,250318.555556,0.394003,0.011400,0.290833,-14.448000,0.210389,117.518111,0.389500,38.333333,5
1,1,26,0.862538,Cats 1983 Broadway Cast,0.441731,287280.000000,0.406808,0.081158,0.315215,-10.690000,0.176212,103.044154,0.268865,30.576923,5
2,1,7,0.856571,Fiddler On The Roof” Motion Picture Chorus,0.348286,328920.000000,0.286571,0.024593,0.325786,-15.230714,0.118514,77.375857,0.354857,34.857143,0
3,1,27,0.884926,Fiddler On The Roof” Motion Picture Orchestra,0.425074,262890.962963,0.245770,0.073587,0.275481,-15.639370,0.123200,88.667630,0.372030,34.851852,0
4,1,7,0.510714,Joseph And The Amazing Technicolor Dreamcoat 1...,0.467143,270436.142857,0.488286,0.009400,0.195000,-10.236714,0.098543,122.835857,0.482286,43.000000,5


In [0]:
# criando novo dataframe com os dados classificados em ordem decrescente pela variável 'count'
artista_ordenado = df_artists.sort_values(by = 'count', ascending=False)
artista_ordenado.head()

,mode,count,acousticness,artists,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key
8367,1,3169,0.984282,Francisco Canaro,0.659676,176986.345850,0.287555,0.537988,0.199492,-12.043738,0.110533,122.743169,0.745212,0.061849,2
28561,1,2422,0.353916,Эрнест Хемингуэй,0.696773,115936.962015,0.188971,0.000027,0.373329,-18.237630,0.926229,111.847911,0.583885,0.066061,11
28560,1,2136,0.297365,Эрих Мария Ремарк,0.693738,121013.185393,0.200289,0.000230,0.195630,-19.259112,0.921277,110.717091,0.536381,0.000000,0
8434,1,1459,0.738865,Frank Sinatra,0.385232,189282.134339,0.236152,0.021148,0.230828,-14.335969,0.049029,109.394801,0.364991,28.658670,5
10714,1,1256,0.985169,Ignacio Corsini,0.545475,154560.711783,0.187842,0.425159,0.182707,-20.862591,0.121051,106.514583,0.638670,0.031847,5


In [0]:
# seleção dos top 10 artistas com mais músicas no conjunto de dados utilizando a variável 'count'
top_artista = artista_ordenado.iloc[0:10]
top_artista

,mode,count,acousticness,artists,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key
8367,1,3169,0.984282,Francisco Canaro,0.659676,176986.345850,0.287555,0.537988,0.199492,-12.043738,0.110533,122.743169,0.745212,0.061849,2
28561,1,2422,0.353916,Эрнест Хемингуэй,0.696773,115936.962015,0.188971,0.000027,0.373329,-18.237630,0.926229,111.847911,0.583885,0.066061,11
28560,1,2136,0.297365,Эрих Мария Ремарк,0.693738,121013.185393,0.200289,0.000230,0.195630,-19.259112,0.921277,110.717091,0.536381,0.000000,0
8434,1,1459,0.738865,Frank Sinatra,0.385232,189282.134339,0.236152,0.021148,0.230828,-14.335969,0.049029,109.394801,0.364991,28.658670,5
10714,1,1256,0.985169,Ignacio Corsini,0.545475,154560.711783,0.187842,0.425159,0.182707,-20.862591,0.121051,106.514583,0.638670,0.031847,5
27109,1,1200,0.990092,Vladimir Horowitz,0.343624,266650.370833,0.118341,0.879898,0.182788,-23.190868,0.043228,94.845258,0.224994,3.694167,1
1682,1,1146,0.917843,Arturo Toscanini,0.315980,309308.737347,0.203463,0.474637,0.280793,-17.185234,0.058348,102.235789,0.283423,0.240838,0
2707,1,1103,0.908776,Billie Holiday,0.571550,185677.068903,0.202251,0.013111,0.217323,-13.221339,0.062274,109.883821,0.498693,16.780598,5
12378,1,1061,0.688296,Johnny Cash,0.620759,164491.506126,0.443462,0.017630,0.239843,-11.663137,0.095992,114.508436,0.672277,26.914232,10
7426,1,1023,0.738890,Elvis Presley,0.493966,157592.318671,0.424834,0.053483,0.245853,-12.921860,0.057361,112.397270,0.617856,34.019550,0


In [0]:
# analisando os resultados com um gráfico de barras
plot_title = 'Top 10 Artistas'
top_artista.plot.bar(x='count', y = 'artists', title = plot_title)

**2. Qual o gênero musical desses top 10 artistas?**

In [0]:
# criando uma lista com os artistas únicos presentes no top 10
lista_artistas = top_artista.artists.unique().to_list()
lista_artistas

Out[13]: ['Francisco Canaro',
 'Эрнест Хемингуэй',
 'Эрих Мария Ремарк',
 'Frank Sinatra',
 'Ignacio Corsini',
 'Vladimir Horowitz',
 'Arturo Toscanini',
 'Billie Holiday',
 'Johnny Cash',
 'Elvis Presley']

In [0]:
artista_genero = df_w_genres.loc[df_w_genres['artists'].isin(lista_artistas)]
artista_genero = artista_genero[['genres', 'artists']]
display(artista_genero)

genres,artists
"classical performance, historic orchestral performance, orchestral performance",Arturo Toscanini
"adult standards, harlem renaissance, jazz blues, lounge, soul, torch song, vocal jazz",Billie Holiday
"rock-and-roll, rockabilly",Elvis Presley
"tango, vintage tango",Francisco Canaro
"adult standards, easy listening, lounge",Frank Sinatra
"tango, vintage tango",Ignacio Corsini
"arkansas country, outlaw country",Johnny Cash
"classical, classical performance, classical piano, russian classical piano",Vladimir Horowitz
,Эрих Мария Ремарк
,Эрнест Хемингуэй


**3. Quais são os top gêneros?**

In [0]:
# cria uma coluna com valor = 1 para contar a quantidade de vezes que cada gênero aparece em nosso conjunto de dados
df_w_genres['qtd'] = 1
df_w_genres.head()

,genres,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count,qtd
0,show tunes,Cats 1981 Original London Cast,0.5901111111111111,0.4672222222222222,250318.5555555556,0.3940033333333333,0.011399851111111107,0.2908333333333333,-14.448000,0.210389,117.518111,0.389500,38.333333,5.0,1.0,9.0,1
1,,Cats 1983 Broadway Cast,0.8625384615384617,0.4417307692307693,287280.0,0.4068076923076923,0.08115826423076923,0.3152153846153846,-10.690000,0.176212,103.044154,0.268865,30.576923,5.0,1.0,26.0,1
2,,Fiddler On The Roof” Motion Picture Chorus,0.8565714285714285,0.34828571428571425,328920.0,0.2865714285714285,0.024592948571428568,0.3257857142857143,-15.230714,0.118514,77.375857,0.354857,34.857143,0.0,1.0,7.0,1
3,,Fiddler On The Roof” Motion Picture Orchestra,0.884925925925926,0.4250740740740739,262890.96296296304,0.2457703703703704,0.07358727925925927,0.2754814814814815,-15.639370,0.123200,88.667630,0.372030,34.851852,0.0,1.0,27.0,1
4,,Joseph And The Amazing Technicolor Dreamcoat 1...,0.5107142857142857,0.4671428571428572,270436.14285714284,0.4882857142857143,0.00940029142857143,0.195,-10.236714,0.098543,122.835857,0.482286,43.000000,5.0,1.0,7.0,1


In [0]:
# Seleciona apenas as colunas de gênero e quantidade
df_w_genres_2 = df_w_genres[['genres', 'qtd']]
df_w_genres_2.head()

,genres,qtd
0,show tunes,1
1,,1
2,,1
3,,1
4,,1


In [0]:
# agrupa os dados por gênero e soma a quantidade de músicas de cada gênero, em seguida ordena os resultados em ordem decrescente
df_w_genres_2_ordenado = df_w_genres_2.groupby('genres').sum().sort_values(by='qtd', ascending=False).reset_index()
df_w_genres_2_ordenado.head(20)

,genres,qtd
0,,9857
1,movie tunes,69
2,show tunes,63
3,hollywood,56
4,orchestral performance,50
5,"broadway, hollywood, show tunes",48
6,"disney, movie tunes",45
7,sleep,42
8,gospel,41
9,"contemporary country, country, country road, m...",41


In [0]:
# seleciona os 10 gêneros mais populares
top_generos = df_w_genres_2_ordenado.loc[0:10]
top_generos

,genres,qtd
0,,9857
1,movie tunes,69
2,show tunes,63
3,hollywood,56
4,orchestral performance,50
5,"broadway, hollywood, show tunes",48
6,"disney, movie tunes",45
7,sleep,42
8,gospel,41
9,"contemporary country, country, country road, m...",41


In [0]:
# seleciona os gêneros do 2º ao 11º mais populares
top_generos_2 = df_w_genres_2_ordenado.loc[1:10]
top_generos_2

,genres,qtd
1,movie tunes,69
2,show tunes,63
3,hollywood,56
4,orchestral performance,50
5,"broadway, hollywood, show tunes",48
6,"disney, movie tunes",45
7,sleep,42
8,gospel,41
9,"contemporary country, country, country road, m...",41
10,classical soprano,40


In [0]:
# Plota um gráfico de barras para os top 10 gêneros
plot_title = 'Top 10 Gêneros'
top_generos_2.plot.bar(x='qtd', y='genres', title=plot_title)

**4. Quais são os artistas que estão nesses top gêneros?**

In [0]:
# cria uma lista para armazenar os nomes dos top gêneros
lista_top_genero = top_generos_2.genres.unique().to_list()
lista_top_genero

Out[46]: ['movie tunes',
 'show tunes',
 'hollywood',
 'orchestral performance',
 'broadway, hollywood, show tunes',
 'disney, movie tunes',
 'sleep',
 'gospel',
 'contemporary country, country, country road, modern country rock',
 'classical soprano']

In [0]:
# criando novo dataframe com somente as linhas presentes na 'lista_top_genero'
genero_artista = df_w_genres.loc[df_w_genres['genres'].isin(lista_top_genero)]
genero_artista.shape

Out[50]: (495, 17)

In [0]:
# lista de artistas que estão no top 10 de gêneros musicais
genero_artista = genero_artista[['genres', 'artists']]
display(genero_artista)

genres,artists
show tunes,Cats 1981 Original London Cast
"broadway, hollywood, show tunes",'Legally Blonde' Ensemble
show tunes,'Legally Blonde' Greek Chorus
"broadway, hollywood, show tunes",Aaron Tveit
"broadway, hollywood, show tunes",Adam Pascal
movie tunes,Adriana Caselotti
classical soprano,Adrianne Pieczonka
classical soprano,Agnes Giebel
gospel,Albertina Walker
"broadway, hollywood, show tunes",Alex Brightman
